This demo shows pretrain `TrxEncoder` with MaskedLanguageModel task.

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

# import logging
import torch
import pytorch_lightning as pl
# import warnings

# warnings.filterwarnings('ignore')
# logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

## Data preproccessing

In [3]:
import os
import pandas as pd

source_data = pd.read_csv(
    'https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true', 
    compression='gzip',
)
source_data.head(2)

client_id  trans_date  small_group  amount_rur
0      33172           6            4      71.463
1      33172           6           35      45.017

In [4]:
from ptls.preprocessing.pandas.pandas_preprocessor import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    col_event_time='trans_date',
    event_time_transformation='none',
    cols_category=['small_group'],
    cols_numerical=['amount_rur'],
    return_records=True,
)

In [5]:
%%time

dataset = preprocessor.fit_transform(source_data)

CPU times: user 1min 22s, sys: 10.7 s, total: 1min 33s
Wall time: 1min 48s


In [6]:
import pickle

with open('preprocessor.p', 'wb') as f:
    pickle.dump(preprocessor, f)

In [7]:
dataset = sorted(dataset, key=lambda x: x['client_id'])

In [8]:
# Load targets
df_target = pd.read_csv(
    'https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true', 
    compression='gzip',
)
df_target.set_index('client_id', inplace=True)
df_target.rename(columns={"bins": "target"}, inplace=True)
df_target.head(5)

target
client_id        
24662           2
1046            0
34089           2
34848           1
47076           3

In [9]:
# Add targets to tdataset
print(dataset[0].keys())

for el in dataset:
    el['target'] = df_target['target'][el['client_id']]

print(dataset[0].keys())

dict_keys(['client_id', 'trans_date', 'event_time', 'small_group', 'amount_rur'])
dict_keys(['client_id', 'trans_date', 'event_time', 'small_group', 'amount_rur', 'target'])


In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=42)
train, valid = train_test_split(train, test_size=4000, random_state=42)

len(train), len(valid), len(test)

(20000, 4000, 6000)

## Embedding training

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (`pytorch_lightning.LightningModule`)
    * data loader (`torch.utils.data.DataLoader`)
    * trainer (`pytorch_lightning.Trainer`)
    
For futher details check https://pytorchlightning.ai/

### Data loader

In [11]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import FeatureFilter
from ptls.frames.bert import MlmDataset
from ptls.frames import PtlsDataModule

mlm_dm = PtlsDataModule(
    train_data=MlmDataset(
        MemoryMapDataset(
            data=train,
        ),
        min_len=100, max_len=128
    ),
    valid_data=MlmDataset(
        MemoryMapDataset(
            data=valid,
            i_filters=[
                FeatureFilter(),
            ],
        ),
        min_len=200, max_len=256
    ),
    train_num_workers=16,
    train_batch_size=128,
)

### Model definition

In [12]:
import torch
from ptls.nn import TrxEncoder, LongformerEncoder
from ptls.frames.bert import MLMPretrainModule
from ptls.nn import PBLinear, PBL2Norm, PBLayerNorm

In [13]:
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'amount_rur': 'identity'},
    embeddings={
        'trans_date': {'in': 800, 'out': 16},
        'small_group': {'in': 250, 'out': 16},
    },
)

trx_encoder = TrxEncoder(**trx_encoder_params)

In [14]:
mlm_module = MLMPretrainModule(
    trx_encoder=torch.nn.Sequential(
        trx_encoder,
        PBLinear(trx_encoder.output_size, 64),
        PBL2Norm(),
    ),
    seq_encoder=LongformerEncoder(
        input_size=64,
        num_attention_heads=1,
        intermediate_size=256,
        num_hidden_layers=2,
        attention_window=32,
        max_position_embeddings=2000,
    ),
    hidden_size=64,
    loss_temperature=20.0,
    
    total_steps=30000,

    replace_proba=0.1,
    neg_count=64,
    
    log_logits=True,
)

### Trainer

In [15]:
import numpy as np
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_steps=1200,  # 12000
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=False,
    callbacks=[
        pl.callbacks.LearningRateMonitor(logging_interval='step'),
        pl.callbacks.ModelCheckpoint(every_n_train_steps=2000, save_top_k=-1),
    ]
)
print(f'loss baseline = {np.log(mlm_module.hparams.neg_count):.3f}')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


loss baseline = 4.159


### Training 

In [16]:
%%time
print(f'logger.version = {trainer.logger.version}')
trainer.fit(mlm_module, mlm_dm)
print(trainer.logged_metrics)

logger.version = 3


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type              | Params
------------------------------------------------------
0 | trx_encoder     | Sequential        | 19.0 K
1 | _seq_encoder    | LongformerEncoder | 253 K 
2 | fn_norm_predict | PBShell           | 0     
3 | loss_fn         | QuerySoftmaxLoss  | 0     
4 | train_mlm_loss  | MeanMetric        | 0     
5 | valid_mlm_loss  | MeanMetric        | 0     
------------------------------------------------------
272 K     Trainable params
0         Non-trainable params
272 K     Total params
1.090     Total estimated model params size (MB)


{'mlm/loss': tensor(3.6981), 'mlm/valid_mlm_loss': tensor(3.6996), 'mlm/train_mlm_loss': tensor(3.7198)}
CPU times: user 3min 18s, sys: 3min 12s, total: 6min 30s
Wall time: 8min 56s


### Save sequence encoder for other experiments

In [17]:
torch.save(mlm_module.trx_encoder.state_dict(), "mlm-emb.pt")

## Finetuning 

### Data module

In [18]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter, FeatureFilter
from ptls.frames.supervised import SeqToTargetDataset
from ptls.frames import PtlsDataModule

def get_dataset(data):
    return SeqToTargetDataset(
        MemoryMapDataset(
            data=data,
            i_filters=[
                FeatureFilter(keep_feature_names='target'),
            ],
        ),
        target_col_name='target',
    )

finetune_dm = PtlsDataModule(
    train_data=get_dataset(train),
    valid_data=get_dataset(valid),
    test_data=get_dataset(test),
    train_num_workers=4,
    train_batch_size=256,
    test_batch_size=128,
)

### Load pretrained TrxEncoder to new model

In [19]:
import torch
import torchmetrics
from functools import partial
from ptls.nn import RnnSeqEncoder, Head
from ptls.frames.supervised import SequenceToTarget

In [20]:
downstream_model = SequenceToTarget(
    seq_encoder=RnnSeqEncoder(
        trx_encoder=torch.nn.Sequential(
            torch.nn.Sequential(
                TrxEncoder(**trx_encoder_params),
                PBLinear(trx_encoder.output_size, 64),
                PBL2Norm(),
            ),
            PBLayerNorm(64),
        ),
        input_size=64,
        hidden_size=64,
        type='gru',
    ),
    head=Head(
        input_size=64,
        use_batch_norm=True,
        objective='classification',
        num_classes=4,
    ),
    loss=torch.nn.NLLLoss(),
    metric_list=torchmetrics.Accuracy(compute_on_step=False),
    pretrained_lr=0.005,
    optimizer_partial=partial(torch.optim.Adam, lr=0.015),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=5, gamma=0.4),
)
downstream_model.seq_encoder.trx_encoder[0].load_state_dict(torch.load('mlm-emb.pt'))

<All keys matched successfully>

In [21]:
trainer_ft = pl.Trainer(
    max_epochs=3, # 32
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=False,
    callbacks=[
        pl.callbacks.LearningRateMonitor(logging_interval='step'),
    ]
)
print(f'logger.version = {trainer_ft.logger.version}')
trainer_ft.fit(downstream_model, finetune_dm)
print(trainer_ft.logged_metrics)
trainer_ft.test(dataloaders=finetune_dm.test_dataloader(), verbose=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | seq_encoder   | RnnSeqEncoder | 44.1 K
1 | head          | Head          | 388   
2 | loss          | NLLLoss       | 0     
3 | train_metrics | ModuleDict    | 0     
4 | valid_metrics | ModuleDict    | 0     
5 | test_metrics  | ModuleDict    | 0     
------------------------------------------------
44.5 K    Trainable params
0         Non-trainable params
44.5 K    Total params
0.178     Total estimated model params size (MB)


logger.version = 4


/home/jovyan/.local/share/virtualenvs/ptls-experiments-Evn9g0Hu/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1446: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /home/jovyan/kireev/demo_fix/pytorch-lifestream/demo/lightning_logs/version_4/checkpoints/epoch=2-step=237.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/jovyan/kireev/demo_fix/pytorch-lifestream/demo/lightning_logs/version_4/checkpoints/epoch=2-step=237.ckpt


{'loss': tensor(0.8163), 'seq_len': tensor(919.), 'y': tensor(1.4688), 'val_loss': tensor(1.0755), 'valid/Accuracy': tensor(0.5213), 'train/Accuracy': tensor(0.5932)}


[{'test/Accuracy': 0.5143333077430725}]

### New model without pretrain

In [22]:
downstream_model = SequenceToTarget(
    seq_encoder=RnnSeqEncoder(
        trx_encoder=TrxEncoder(**trx_encoder_params),
        hidden_size=64,
        type='gru',
    ),
    head=Head(
        input_size=64,
        use_batch_norm=False,
        objective='classification',
        num_classes=4,
    ),
    loss=torch.nn.NLLLoss(),
    metric_list=torchmetrics.Accuracy(compute_on_step=False),
    optimizer_partial=partial(torch.optim.Adam, lr=0.015),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=5, gamma=0.5),
)

In [23]:
trainer_ft = pl.Trainer(
    max_epochs=3,  # 32
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=False,
)
print(f'logger.version = {trainer_ft.logger.version}')
trainer_ft.fit(downstream_model, finetune_dm)
print(trainer_ft.logged_metrics)
trainer_ft.test(dataloaders=finetune_dm.test_dataloader())

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | seq_encoder   | RnnSeqEncoder | 35.9 K
1 | head          | Head          | 260   
2 | loss          | NLLLoss       | 0     
3 | train_metrics | ModuleDict    | 0     
4 | valid_metrics | ModuleDict    | 0     
5 | test_metrics  | ModuleDict    | 0     
------------------------------------------------
36.1 K    Trainable params
0         Non-trainable params
36.1 K    Total params
0.145     Total estimated model params size (MB)


logger.version = 5


Restoring states from the checkpoint path at /home/jovyan/kireev/demo_fix/pytorch-lifestream/demo/lightning_logs/version_5/checkpoints/epoch=2-step=237.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/jovyan/kireev/demo_fix/pytorch-lifestream/demo/lightning_logs/version_5/checkpoints/epoch=2-step=237.ckpt


{'loss': tensor(0.7133), 'seq_len': tensor(882.5625), 'y': tensor(1.2188), 'val_loss': tensor(0.9522), 'valid/Accuracy': tensor(0.5645), 'train/Accuracy': tensor(0.5745)}
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test/Accuracy         0.5786666870117188
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/Accuracy': 0.5786666870117188}]